In [ ]:
import torch
# import torch.nn as nn
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader, Batch
from torch_geometric.nn import SAGEConv, GAE
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split
from torch.nn import Embedding
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gensim.downloader as api

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
train_data = pd.read_csv("./prepro_train_data.csv")

# Make Graph
## Category Embedding

### Location

In [ ]:
### Location

# LabelEncoder를 사용해 위치 정보를 정수로 변환
le = LabelEncoder()
train_data['Location_encoded'] = le.fit_transform(train_data['Location'])

# 임베딩 레이어 초기화
embedding_layer = Embedding(num_embeddings=151, embedding_dim=10)

# 위치 정보를 10차원 벡터로 변환
location_embeddings = embedding_layer(torch.unsqueeze(torch.tensor(train_data['Location_encoded'].values, dtype=torch.long), dim=1))

# 결과를 NumPy array로 변환
location_embeddings = location_embeddings.detach().numpy().squeeze()

In [ ]:
print("location_embeddings.shape >>> ",location_embeddings.shape)
print(location_embeddings)

### Book-Title

In [ ]:
import fasttext

# # 사전 훈련된 FastText 모델 다운로드
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gunzip cc.en.300.bin.gz

In [ ]:
# FastText 모델 로드
fasttext_model = fasttext.load_model("cc.en.300.bin")

In [ ]:
def get_title_embedding_fasttext(title):
    words = title.split()
    if len(words) == 0:
        return np.zeros(300)
    embeddings = [fasttext_model.get_word_vector(word) for word in words]
    return np.mean(embeddings, axis=0)

In [ ]:
title_embeddings = train_data['Book-Title'].apply(get_title_embedding_fasttext).tolist()

In [ ]:
empty_titles_count = 0
for title in train_data['Book-Title']:
    words = title.split()
    if not words:
        empty_titles_count += 1

print(f"빈 문자열이거나 토큰화된 단어가 없는 책 제목의 개수: {empty_titles_count}")

In [ ]:
# 임베딩이 numpy 배열이 아니거나 형태가 (300,)이 아닌 경우 해당 인덱스, 책 제목, 및 임베딩을 출력

for idx, emb in enumerate(title_embeddings):
    if not isinstance(emb, np.ndarray) or emb.shape != (300,):
        print(f"Index: {idx}, Title: {train_data['Book-Title'][idx]}, Embedding: {emb}")

In [ ]:
from sklearn.decomposition import PCA

# PCA 객체를 생성
pca = PCA(n_components=30)

# 평균 임베딩 벡터로 구성된 리스트를 NumPy 배열로 변환
title_embeddings_array = np.array(title_embeddings)

# 차원 축소
reduced_title_embeddings = pca.fit_transform(title_embeddings_array)

In [ ]:
print("reduced_title_embeddings.shape >>> ",reduced_title_embeddings.shape)
print(reduced_title_embeddings)

### Publisher

In [ ]:
### Publisher
# LabelEncoder를 사용해 위치 정보를 정수로 변환
le = LabelEncoder()
train_data['Publisher_encoded'] = le.fit_transform(train_data['Publisher'])

# 임베딩 레이어 초기화
embedding_layer = Embedding(num_embeddings=3689, embedding_dim=10)

# 위치 정보를 10차원 벡터로 변환
Publisher_embeddings = embedding_layer(torch.unsqueeze(torch.tensor(train_data['Publisher_encoded'].values, dtype=torch.long), dim=1))

# 결과를 NumPy array로 변환
Publisher_embeddings = Publisher_embeddings.detach().numpy().squeeze()

In [ ]:
print("Publisher_embeddings.shape >>> ",Publisher_embeddings.shape)
print(Publisher_embeddings)

### User-ID & Book-ID